In [1]:
import json
import requests
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
def fetch_all():
    coin_history={}
    for coin in coins:
        coin_history[coin] = fetch_history(coin)
    return coin_history

def fetch_history(coin):
  endpoint_url = "https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym=USD&limit={:d}".format(coin, days_ago_to_fetch)
  res = requests.get(endpoint_url)
  hist = pd.DataFrame(json.loads(res.content)['Data'])
  hist = index_history(hist)
  hist = filter_history_by_date(hist)
  return hist

def index_history(hist):
  # index by date so we can easily filter by a given timeframe
  hist = hist.set_index('time')
  hist.index = pd.to_datetime(hist.index, unit='s')
  return hist

def filter_history_by_date(hist):
  result = hist # customize here
  # result = hist[hist.index.year >= 2017]
  # result = result[result.index.day == 1] # every first of month, etc.
  return result

def combine_data(ch):
    ndf = pd.DataFrame()
    for coin in coins:
        ndf =pd.concat([ndf, ch[coin]['close']], axis=1, sort=False)
    
    ndf.columns = coins
    
    for coin in coins:
        ndf[coin+'_chng']=ndf[coin].pct_change()*100
        
    for i in range(0, len(coins)):
        coin=coins[i]
        #print( ndf.iat[0, len(coins)+i])
        ndf.iat[0, len(coins)+i]=0
        #print( 'tada', ndf.iloc[0][coin+'_chng'] )
        
    return ndf

def get_extremum(ch_d, orig):
    min_v = {'col':'None', 'offset': 100}
    max_v = {'col':'None', 'offset':-100}
    
    #print('#'*20)
    #print('@',ch_d)
    
    for coin in coins:
        #print(coin, orig[coin+'_share'], ch_d[coin+'_share'],  ch_d[coin+'_share']-orig[coin+'_share'])
        
        new_offset=ch_d[coin+'_share']-orig[coin+'_share']
        if max_v['offset'] < new_offset and new_offset != 0:
            max_v['col']=coin
            max_v['offset']=new_offset
        
        if min_v['offset'] > new_offset and new_offset != 0:
            min_v['col']=coin
            min_v['offset']=new_offset
    
    #print('# min', min_v, 'max', max_v)
    return min_v, max_v

def gen_cv(cur, loc_pf):
    cv = {}
    for coin in coins:
        #print('$$ loc_pf', loc_pf)
        #print('$$ cur', cur, type(cur))
        #print('$$ coin', coin)
        #print('$$ cur coin', cur[coin])
        #print('='*20)
        cv[coin+'_v']=loc_pf[coin+'_tokens'] * cur[coin]

    #calculate current portfolio, sum(coin_values) 
    sm = 0
    for coin in coins: sm += cv[coin+'_v']
    
    #calculate each coin share
    for coin in coins: cv[coin+'_sh'] = cv[coin+'_v']/sm
    
    #calculate share change in portfolio: share_current_value - portfolio_latest_state
    for coin in coins: cv[coin+'_sh_chng'] = cv[coin+'_sh']-loc_pf[coin+'_sh']
            
    #for coin in coins: print('{} sm:{:.4f} orig_s:{:.4f} cur:_s{:.4f} chng:{:.4f}'.format(
    #    coin, sm, pf[coin+'_sh_orig'], cv[coin+'_sh'], cv[coin+'_sh_chng']*100))
    #print(coin)
    return cv

def recalculate_pf_values(cur, loc_pf):
    for coin in coins:
        loc_pf[coin+'_v']=loc_pf[coin+'_tokens']*cur[coin]
    return cur, loc_pf

def main_test(i_threshold, i_pf, i_ndf):
    cva=[]
    n = 0
    ctr=0
    #print('#ndf shape', i_ndf.shape[0])
    while n < i_ndf.shape[0]:
        cur = i_ndf.iloc[n]
        #print('first print', cur)
        cur, pf = recalculate_pf_values(cur, i_pf)
        #print('second print', cur)

        inspect_loop=True
        added=False
        while inspect_loop:
            #get current token values and their portfolio share
            cv = gen_cv(cur, i_pf)
            if added==False:
                cva.append(cv)
                #print('# the cva', cva)
                added=True

            #print('#n, cv', n, cv)
            max_thresh=False
            min_thresh=False

            #rebalance coins until portfolio chanhe less then threshold
            for coin in coins:
                #if calculated current values more then threshold: reballance
                if cv[coin+'_sh_chng']*100 > i_threshold:
                    max_thresh=True
                    #print('#%'*20, coin, cv[coin+'_sh_chng']*100)

                    #get changed values with names and sort them
                    flds = []
                    for rec in cv:  
                        if '_sh_chng' in rec: flds.append({'chng':cv[rec], 'nm':rec.replace('_sh_chng','')})
                    sflds=sorted(flds, key = lambda k:k['chng'], reverse=False)
                    #print(sflds)

                    #nm is the lowest current absolute change of pf value
                    nm = sflds[0]['nm']
                    #for rec in i_pf:
                    #    if coin in rec or nm in rec: print(rec, i_pf[rec])


                    excess_tokens = i_pf[coin+'_tokens']*cv[coin+'_sh_chng']
                    excess_value = cur[coin]*excess_tokens
                    shortage_tokens = excess_value/cur[nm]

                    i_pf[coin+'_tokens'] -= excess_tokens * (1-0.001)               
                    i_pf[nm+'_tokens'] += shortage_tokens * (1-0.001)
                    cur, pf = recalculate_pf_values(cur, i_pf)

                    #print('&'*20)
                    #for rec in i_pf: 
                    #    if coin in rec or nm in rec: 
                    #        print(rec,i_pf[rec])
                    ctr+=1
                    #if ctr>10:
                    #    n=1000000
                    #    inspect_loop=False

                #if calculated current values more then threshold: reballance    
                if cv[coin+'_sh_chng']*100 < -i_threshold:
                    min_thresh=True
                    #print('#'*20, coin, cv[coin+'_sh_chng']*100)
                    flds = []
                    for rec in cv:  
                        if '_sh_chng' in rec: flds.append({'chng':cv[rec], 'nm':rec.replace('_sh_chng','')})
                    sflds=sorted(flds, key = lambda k:k['chng'], reverse=True)
                    #print(sflds)

                    nm = sflds[0]['nm']
                    #for rec in i_pf:
                    #    if coin in rec or nm in rec: print('pf:', rec, i_pf[rec])

                    shortage_tokens = i_pf[coin+'_tokens']*cv[coin+'_sh_chng']
                    shortage_value = shortage_tokens*cur[coin]
                    #print('sht_tk', shortage_tokens, 'sht_vl', shortage_value)
                    target_value = shortage_value*0.2
                    excess_tokens = target_value/cur[nm]
                    #print('tvl', target_value, 'excc tkn', excess_tokens, nm)
                    #print('prc',cur[coin])
                    i_pf[nm+'_tokens']-=abs(excess_tokens) * (1-0.001)
                    i_pf[coin+'_tokens']+=abs(shortage_tokens) * (1-0.001)


                    #print('chng:', cv[nm+'_sh_chng'], 'pf', i_pf[nm+'_tokens'])
                    #print('excess:{:.8f} ex_vl:{:.8f} sht_tkn:{:.8f}'.format(
                    #    excess_tokens, 
                    #    excess_value, 
                    #    shortage_tokens))

                    cur, i_pf = recalculate_pf_values(cur, i_pf)

                    #print('%'*20)
                    #for rec in i_pf: 
                    #    if coin in rec or nm in rec: 
                    #        print(rec, i_pf[rec])

                    ctr+=1
                    #if ctr>10:
                    #    n=1000000
                    #    inspect_loop=False

            #print('ILOOP', inspect_loop, n, max_thresh, min_thresh)
            if max_thresh==False and min_thresh==False: inspect_loop=False
        #print('#', n)
        n+=1    
    return cva, pf

# GLOBAL
avg=120
coins = ["BTC", "ETH", "BNB"]
days_ago_to_fetch = 250 + avg  # see also filter_history_by_date()

hist_length = 0
#average_returns = {}
#cumulative_returns = {}

coin_history = fetch_all()
hist_length = len(coin_history[coins[0]])
ndf = combine_data(coin_history)
print(ndf.head())

my_pf={}
my_pf['sum']=0
for rec in coins:
    my_pf[rec+'_v']=100
for coin in coins: my_pf[coin+'_orig_v'] = my_pf[coin+'_v']
for coin in coins: my_pf[coin+'_tokens'] = my_pf[coin+'_v']/ndf.iloc[0][coin]
for coin in coins: my_pf[coin+'_orig_tokens'] = my_pf[coin+'_v']/ndf.iloc[0][coin]
for coin in coins: my_pf['sum'] += my_pf[coin+'_v']
my_pf['sum_orig'] = my_pf['sum']    
for coin in coins: my_pf[coin+'_sh_orig'] = my_pf[coin+'_v']/my_pf['sum_orig']
for coin in coins: my_pf[coin+'_sh'] = my_pf[coin+'_v']/my_pf['sum']

#my_pf


                BTC     ETH    BNB  BTC_chng   ETH_chng   BNB_chng
time                                                              
2018-04-24  9655.77  703.35  15.16  0.000000   0.000000   0.000000
2018-04-25  8873.62  617.73  13.40 -8.100338 -12.173171 -11.609499
2018-04-26  9282.12  661.45  14.62  4.603533   7.077526   9.104478
2018-04-27  8938.47  643.33  14.26 -3.702279  -2.739436  -2.462380
2018-04-28  9351.47  683.02  15.16  4.620478   6.169462   6.311360


In [3]:
def prepare(prep, pf):
    for coin in coins:
        pf[coin+'_avg']=np.mean(prep[coin])
    return pf

def run_with_params(i_beg, i_end, i_threshold):
    my_pf = prepare(ndf.iloc[i_beg:i_end], {})

    my_pf['sum']=0
    for rec in coins:
        my_pf[rec+'_v']=100

    for coin in coins: my_pf[coin+'_orig_v'] = my_pf[coin+'_v']
    #for coin in coins: my_pf[coin+'_tokens'] = my_pf[coin+'_v']/ndf.iloc[0][coin]
    #for coin in coins: my_pf[coin+'_orig_tokens'] = my_pf[coin+'_v']/ndf.iloc[0][coin]
    for coin in coins: my_pf[coin+'_tokens'] = my_pf[coin+'_v']/my_pf[coin+'_avg']
    for coin in coins: my_pf[coin+'_orig_tokens'] = my_pf[coin+'_v']/my_pf[coin+'_avg']
    
    for coin in coins: my_pf['sum'] += my_pf[coin+'_v']
    my_pf['sum_orig'] = my_pf['sum']    
    for coin in coins: my_pf[coin+'_sh_orig'] = my_pf[coin+'_v']/my_pf['sum_orig']
    for coin in coins: my_pf[coin+'_sh'] = my_pf[coin+'_v']/my_pf['sum']

    cva, pf = main_test(i_threshold, my_pf, ndf.iloc[i_end:hist_length-1])

    pf['sum']=0
    for coin in coins: pf['sum'] += pf[coin+'_v']
    print('='*20)
    ndf.tail()
    #ndf.head()
    return {'beg':i_beg, 'end':i_end, 'thr':i_threshold, 'pf':pf, 'len_cva':len(cva)}

In [8]:
import datetime
rslt_arr = []
for thr in range(1,10):
    for f in range(1,120):
        print(datetime.datetime.now(), thr, f)
        rslt = run_with_params(f, 120, thr)
        rslt['sum'] = rslt['pf']['sum']
        rslt_arr.append(rslt)

2019-04-29 20:21:35.773107 1 1
2019-04-29 20:21:35.861300 1 2
2019-04-29 20:21:35.944599 1 3
2019-04-29 20:21:36.030412 1 4
2019-04-29 20:21:36.114167 1 5
2019-04-29 20:21:36.198124 1 6
2019-04-29 20:21:36.283702 1 7
2019-04-29 20:21:36.367813 1 8
2019-04-29 20:21:36.450200 1 9
2019-04-29 20:21:36.534583 1 10
2019-04-29 20:21:36.618785 1 11
2019-04-29 20:21:36.704554 1 12
2019-04-29 20:21:36.791674 1 13
2019-04-29 20:21:36.879010 1 14
2019-04-29 20:21:36.963439 1 15
2019-04-29 20:21:37.047116 1 16
2019-04-29 20:21:37.125864 1 17
2019-04-29 20:21:37.208231 1 18
2019-04-29 20:21:37.294507 1 19
2019-04-29 20:21:37.375717 1 20
2019-04-29 20:21:37.458533 1 21
2019-04-29 20:21:37.542144 1 22
2019-04-29 20:21:37.624357 1 23
2019-04-29 20:21:37.707259 1 24
2019-04-29 20:21:37.792747 1 25
2019-04-29 20:21:37.877515 1 26
2019-04-29 20:21:37.960663 1 27
2019-04-29 20:21:38.045833 1 28
2019-04-29 20:21:38.127265 1 29
2019-04-29 20:21:38.209058 1 30
2019-04-29 20:21:38.292728 1 31
2019-04-29 20:21:

2019-04-29 20:21:48.518181 2 37
2019-04-29 20:21:48.588850 2 38
2019-04-29 20:21:48.662445 2 39
2019-04-29 20:21:48.738413 2 40
2019-04-29 20:21:48.812706 2 41
2019-04-29 20:21:48.885772 2 42
2019-04-29 20:21:48.959693 2 43
2019-04-29 20:21:49.032620 2 44
2019-04-29 20:21:49.105492 2 45
2019-04-29 20:21:49.181391 2 46
2019-04-29 20:21:49.253745 2 47
2019-04-29 20:21:49.332195 2 48
2019-04-29 20:21:49.407100 2 49
2019-04-29 20:21:49.483264 2 50
2019-04-29 20:21:49.556818 2 51
2019-04-29 20:21:49.631363 2 52
2019-04-29 20:21:49.704122 2 53
2019-04-29 20:21:49.776885 2 54
2019-04-29 20:21:49.849835 2 55
2019-04-29 20:21:49.923452 2 56
2019-04-29 20:21:49.994839 2 57
2019-04-29 20:21:50.067691 2 58
2019-04-29 20:21:50.139447 2 59
2019-04-29 20:21:50.213634 2 60
2019-04-29 20:21:50.284936 2 61
2019-04-29 20:21:50.361695 2 62
2019-04-29 20:21:50.434037 2 63
2019-04-29 20:21:50.508666 2 64
2019-04-29 20:21:50.583043 2 65
2019-04-29 20:21:50.658305 2 66
2019-04-29 20:21:50.732332 2 67
2019-04-

2019-04-29 20:21:59.725650 3 73
2019-04-29 20:21:59.795130 3 74
2019-04-29 20:21:59.862570 3 75
2019-04-29 20:21:59.938030 3 76
2019-04-29 20:22:00.012493 3 77
2019-04-29 20:22:00.088592 3 78
2019-04-29 20:22:00.161894 3 79
2019-04-29 20:22:00.233554 3 80
2019-04-29 20:22:00.303642 3 81
2019-04-29 20:22:00.378721 3 82
2019-04-29 20:22:00.450874 3 83
2019-04-29 20:22:00.520132 3 84
2019-04-29 20:22:00.589671 3 85
2019-04-29 20:22:00.660681 3 86
2019-04-29 20:22:00.732269 3 87
2019-04-29 20:22:00.804784 3 88
2019-04-29 20:22:00.871929 3 89
2019-04-29 20:22:00.942481 3 90
2019-04-29 20:22:01.012836 3 91
2019-04-29 20:22:01.083478 3 92
2019-04-29 20:22:01.154414 3 93
2019-04-29 20:22:01.224812 3 94
2019-04-29 20:22:01.295407 3 95
2019-04-29 20:22:01.363425 3 96
2019-04-29 20:22:01.436843 3 97
2019-04-29 20:22:01.507331 3 98
2019-04-29 20:22:01.581010 3 99
2019-04-29 20:22:01.653023 3 100
2019-04-29 20:22:01.722038 3 101
2019-04-29 20:22:01.790179 3 102
2019-04-29 20:22:01.857501 3 103
2019

2019-04-29 20:22:10.546252 4 111
2019-04-29 20:22:10.613076 4 112
2019-04-29 20:22:10.678660 4 113
2019-04-29 20:22:10.746731 4 114
2019-04-29 20:22:10.813294 4 115
2019-04-29 20:22:10.880135 4 116
2019-04-29 20:22:10.944349 4 117
2019-04-29 20:22:11.013408 4 118
2019-04-29 20:22:11.078662 4 119
2019-04-29 20:22:11.143490 5 1
2019-04-29 20:22:11.208491 5 2
2019-04-29 20:22:11.276157 5 3
2019-04-29 20:22:11.341982 5 4
2019-04-29 20:22:11.407660 5 5
2019-04-29 20:22:11.472912 5 6
2019-04-29 20:22:11.539577 5 7
2019-04-29 20:22:11.605459 5 8
2019-04-29 20:22:11.670574 5 9
2019-04-29 20:22:11.739204 5 10
2019-04-29 20:22:11.808846 5 11
2019-04-29 20:22:11.873782 5 12
2019-04-29 20:22:11.939463 5 13
2019-04-29 20:22:12.005324 5 14
2019-04-29 20:22:12.073655 5 15
2019-04-29 20:22:12.139972 5 16
2019-04-29 20:22:12.206878 5 17
2019-04-29 20:22:12.275039 5 18
2019-04-29 20:22:12.344319 5 19
2019-04-29 20:22:12.411252 5 20
2019-04-29 20:22:12.496223 5 21
2019-04-29 20:22:12.573342 5 22
2019-04-

2019-04-29 20:22:21.031322 6 29
2019-04-29 20:22:21.102230 6 30
2019-04-29 20:22:21.172847 6 31
2019-04-29 20:22:21.241899 6 32
2019-04-29 20:22:21.313327 6 33
2019-04-29 20:22:21.385969 6 34
2019-04-29 20:22:21.456941 6 35
2019-04-29 20:22:21.524486 6 36
2019-04-29 20:22:21.589339 6 37
2019-04-29 20:22:21.654097 6 38
2019-04-29 20:22:21.721001 6 39
2019-04-29 20:22:21.786493 6 40
2019-04-29 20:22:21.854090 6 41
2019-04-29 20:22:21.925115 6 42
2019-04-29 20:22:21.995728 6 43
2019-04-29 20:22:22.060720 6 44
2019-04-29 20:22:22.125410 6 45
2019-04-29 20:22:22.191529 6 46
2019-04-29 20:22:22.256322 6 47
2019-04-29 20:22:22.322194 6 48
2019-04-29 20:22:22.389006 6 49
2019-04-29 20:22:22.457493 6 50
2019-04-29 20:22:22.524484 6 51
2019-04-29 20:22:22.604746 6 52
2019-04-29 20:22:22.669811 6 53
2019-04-29 20:22:22.736923 6 54
2019-04-29 20:22:22.808957 6 55
2019-04-29 20:22:22.878015 6 56
2019-04-29 20:22:22.944744 6 57
2019-04-29 20:22:23.012735 6 58
2019-04-29 20:22:23.078478 6 59
2019-04-

2019-04-29 20:22:31.666916 7 67
2019-04-29 20:22:31.735182 7 68
2019-04-29 20:22:31.800455 7 69
2019-04-29 20:22:31.864972 7 70
2019-04-29 20:22:31.930653 7 71
2019-04-29 20:22:31.997894 7 72
2019-04-29 20:22:32.062451 7 73
2019-04-29 20:22:32.127487 7 74
2019-04-29 20:22:32.195834 7 75
2019-04-29 20:22:32.265199 7 76
2019-04-29 20:22:32.331302 7 77
2019-04-29 20:22:32.397018 7 78
2019-04-29 20:22:32.462714 7 79
2019-04-29 20:22:32.527425 7 80
2019-04-29 20:22:32.592862 7 81
2019-04-29 20:22:32.659674 7 82
2019-04-29 20:22:32.726423 7 83
2019-04-29 20:22:32.790948 7 84
2019-04-29 20:22:32.855772 7 85
2019-04-29 20:22:32.919895 7 86
2019-04-29 20:22:32.984040 7 87
2019-04-29 20:22:33.047713 7 88
2019-04-29 20:22:33.111356 7 89
2019-04-29 20:22:33.176298 7 90
2019-04-29 20:22:33.240510 7 91
2019-04-29 20:22:33.304110 7 92
2019-04-29 20:22:33.368735 7 93
2019-04-29 20:22:33.434038 7 94
2019-04-29 20:22:33.498593 7 95
2019-04-29 20:22:33.560161 7 96
2019-04-29 20:22:33.623155 7 97
2019-04-

2019-04-29 20:22:42.221356 8 106
2019-04-29 20:22:42.290810 8 107
2019-04-29 20:22:42.357972 8 108
2019-04-29 20:22:42.424165 8 109
2019-04-29 20:22:42.491380 8 110
2019-04-29 20:22:42.556701 8 111
2019-04-29 20:22:42.621735 8 112
2019-04-29 20:22:42.688903 8 113
2019-04-29 20:22:42.752899 8 114
2019-04-29 20:22:42.818910 8 115
2019-04-29 20:22:42.884122 8 116
2019-04-29 20:22:42.948938 8 117
2019-04-29 20:22:43.013958 8 118
2019-04-29 20:22:43.077606 8 119
2019-04-29 20:22:43.141390 9 1
2019-04-29 20:22:43.206781 9 2
2019-04-29 20:22:43.269956 9 3
2019-04-29 20:22:43.334935 9 4
2019-04-29 20:22:43.398060 9 5
2019-04-29 20:22:43.462415 9 6
2019-04-29 20:22:43.527706 9 7
2019-04-29 20:22:43.590547 9 8
2019-04-29 20:22:43.654999 9 9
2019-04-29 20:22:43.722560 9 10
2019-04-29 20:22:43.786952 9 11
2019-04-29 20:22:43.852563 9 12
2019-04-29 20:22:43.917371 9 13
2019-04-29 20:22:43.985589 9 14
2019-04-29 20:22:44.053040 9 15
2019-04-29 20:22:44.116205 9 16
2019-04-29 20:22:44.182336 9 17
201

In [16]:
rsltd=[]
for rec in rslt_arr:
    vls = {}
    for d in rec['pf']:
        if 'avg' in d:
            vls[d]=round(rec['pf'][d],3)
        if 'orig_tokens' in d:
            vls[d]=round(rec['pf'][d],3)    
    #print(rec['beg'], rec['end'], round(rec['sum'],4), rec['thr'])
    rsltd.append( {'beg':rec['beg'], 'end':rec['end'], 'sum':round(rec['sum'],4), 'thr':rec['thr']} )
pd.DataFrame(rsltd).sort_values(by=['sum'], ascending=False)

,beg,end,sum,thr
117,118,120,454.2842,1
118,119,120,448.9314,1
111,112,120,446.6901,1
110,111,120,446.6587,1
115,116,120,446.2762,1
116,117,120,445.3233,1
112,113,120,444.2812,1
113,114,120,442.1768,1
114,115,120,440.1839,1
108,109,120,436.9612,1
